In [ ]:
import gym
from gym import spaces
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import t, mode
from tqdm import tqdm

In [ ]:
def print_gridworld_with_policy(agent, figsize=(6, 6), title="notitle"):
    plt.figure(figsize=figsize)

    for state in range(agent.env.state_space.n):
        col, row = agent.env.index_to_state(state)

        plt.gca().add_patch(
            plt.Rectangle((col, agent.env.grid_size - row - 1), 1, 1, facecolor='white', edgecolor='black'))

        for a in range(agent.env.action_space.n):
            prob = np.exp(agent.policy_params[state, a]) / np.sum(np.exp(agent.policy_params[state]))
            # Calculate the center coordinates of the state cell
            center_x = col + 0.5
            center_y = agent.env.grid_size - row - 0.5

            # Calculate the arrow starting coordinates relative to the center
            arrow_start_x = center_x
            arrow_start_y = center_y

            # Calculate the arrow ending coordinates relative to the center based on the action
            if a == 0:  # Up arrow
                arrow_end_x = center_x
                arrow_end_y = center_y + prob * 0.3  # Adjust the scaling factor to control the arrow length
            elif a == 1:  # Down arrow
                arrow_end_x = center_x
                arrow_end_y = center_y - prob * 0.3
            elif a == 2:  # Left arrow
                arrow_end_x = center_x - prob * 0.3
                arrow_end_y = center_y
            elif a == 3:  # Right arrow
                arrow_end_x = center_x + prob * 0.3
                arrow_end_y = center_y

            # Calculate the arrowhead size based on the value
            head_width = prob * 0.2  # Adjust the scaling factor to control the arrowhead width
            head_length = prob * 0.2  # Adjust the scaling factor to control the arrowhead length

            # Draw the arrow
            arrow_thickness = 1  # Adjust the scaling factor to control the arrow thickness
            plt.arrow(arrow_start_x, arrow_start_y, arrow_end_x - arrow_start_x, arrow_end_y - arrow_start_y,
                      head_width=head_width, head_length=head_length, fc='black', ec='black', linewidth=arrow_thickness)

    plt.axis('scaled')
    plt.axis('off')
    plt.show()

def print_heatmap(agent, d_t, title):
    '''
    This method serves the purpose of printing a heatmap of the state distribution given a trajectory of states drawn by
    an agent acting inside an environment.
    :param agent: the agent that acted in the environment;
    :param d_t: the state visitation distribution.
    '''
    # Create the subplot
    ax = plt.gca()
    # Check corridor
    if hasattr(agent.env, 'length_corridor'):
        heatmap_data = np.full((agent.env.grid_size + agent.env.length_corridor, agent.env.grid_size), np.nan)
    else:
        heatmap_data = np.full((agent.env.grid_size, agent.env.grid_size), np.nan)
    # Prepare data for plotting
    for index in range(agent.env.state_space.n):
        col, row = agent.env.index_to_state(index)
        heatmap_data[row][col] = d_t[index]
    # Plot the heatmap
    sns.heatmap(data=heatmap_data, annot=True, ax=ax)
    ax.set_title(title)
    plt.show()

def plot_graph(n_run, n_episodes, list_entropies, list_true_entropies, confidence):
    # Initialize the plotting vectors
    fig, ax = plt.subplots(figsize=(30, 10))
    plt.title("Big number of episodes normal learning rate, normal gaussian")
    entropies_means = []
    true_entropies_means = []
    under_line = []
    over_line = []

    for i in range(n_episodes):
        entropies_mean = np.mean(list_entropies[i])
        true_entropies_mean = np.mean(list_true_entropies[i])
        entropies_std = np.std(list_entropies[i])
        freedom_deg = n_run - 1
        t_crit = np.abs(t.ppf((1 - confidence) / 2, freedom_deg))
        entropies_means.append(entropies_mean)
        true_entropies_means.append(true_entropies_mean)
        under_line.append(entropies_mean - entropies_std * t_crit / np.sqrt(n_run))
        over_line.append(entropies_mean + entropies_std * t_crit / np.sqrt(n_run))

    ax.clear()
    ax.plot(entropies_means, label='Believed Entropy')
    ax.plot(true_entropies_means, label='True Entropy')
    ax.fill_between(np.arange(n_episodes), under_line, over_line, color='b', alpha=0.1)
    ax.set_xlabel('Episode')
    ax.set_ylabel('Entropy')
    ax.legend()
    fig.canvas.draw()
    return entropies_means, true_entropies_means

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Corridor Gridworld MDP

In [ ]:
class GridworldEnv(gym.Env):
    metadata = {'render.modes': ['human']}

    """
    This class implements a custom made Gym environment of a simple Gridworld, a NxN matrix where the agent starts from a starting position I, with an
    additional corridor situated in the right bottom corner.

    The set of states S = {[x,y]| x,y ∈ [0, ..., grid_size]} or {}, represent the possible positions in the environment.
    The set of actions A = [up, down, left, right].
    The transition to a new state T(s_t+1 | s_t, a_t).
    The reward R(s, a) = {1 if s == G else -0.1}.

    In this version the initial state is the position 0 ([0, 0]), I = [0, 0]. The goal state is the position 24 ([4, 4]) G = [4,4].

    Args:
    - grid_size: the size in height and length of the grid, N of the NxN matrix.
    - time_horizon: the maximum number of time steps the agent can take to get to the goal. If set to -1 the time horizon is ∞.
    - prob: the probability with which the environment takes the chosen action. If set to 0 the actions taken by the agent are deterministic.
    - length_corridor: the length of the corridor
    """

    def __init__(self, grid_size=5, time_horizon=-1, prob=0.1, randomize=0, length_corridor=3):
        self.grid_size = grid_size
        self.length_corridor = length_corridor
        self.state_space = spaces.Discrete(self.grid_size ** 2 + self.length_corridor)
        self.action_space = spaces.Discrete(4)
        self.reward_range = (-0.1, 1.0)
        self.goal = (grid_size-1, grid_size-1 + length_corridor)
        self.current_pos = (0, length_corridor + grid_size - 1)
        self.done = False
        self.time_horizon = time_horizon
        self.steps_taken = 0
        self.prob = prob
        self.transition_matrix = self._build_transition_matrix()
        self.randomize = randomize
        if self.randomize == 1:
            self.current_pos = self.index_to_state(np.random.randint(0, self.state_space.n))
        else:
            self.current_pos = (0, 0)

    def _build_transition_matrix(self):
        '''
        This method builds the transition matrix for the MDP, T(s'|a, s).
        The function should be used with the following order of operands:
         - first parameter: s, the state where the agent takes the action in
         - second parameter: s', the state where the agent arrives taking action a from state s
         - third parameter: a, the action taken by the agent
        '''
        transition_matrix = np.zeros((self.state_space.n, self.state_space.n, self.action_space.n))
        # For every state (i,j)
        for index in range(self.state_space.n):
            s = self.index_to_state(index)
            # For every action 'a' chosen as action from the agent
            for a in range(self.action_space.n):
                # For all actions 'a_'
                for a_ in range(self.action_space.n):
                    # Calculate the probability of
                    prob = 1 - self.prob if a_ == a else self.prob / (self.action_space.n - 1)
                    s_ = self._sample_new_position(a_, s)
                    transition_matrix[self.state_to_index(s_), self.state_to_index(s), a] += prob
        return transition_matrix

    def _sample_new_position(self, action, state):
        if action == 0:  # up
            if state[1] >= self.grid_size:
                new_pos = (state[0], state[1] - 1) # if you are in the corridor you just go one y less
            else:
                new_pos = (state[0], max(0, state[1]-1))
        elif action == 1:  # down
            if state == (self.grid_size - 1, self.grid_size - 1) or state[1] >= self.grid_size:
                new_pos = (state[0], min(self.grid_size + self.length_corridor - 1, state[1] + 1)) # if you are in the corridor, or you are entering the corridor, you add one unless you are at the end of the corridor
            else:
                new_pos = (state[0], min(self.grid_size-1, state[1]+1))
        elif action == 2:  # left
            if state[1] >= self.grid_size:
                new_pos = state # if you are in the corridor you don't move
            else:
                new_pos = (max(0, state[0]-1), state[1])
        elif action == 3:  # right
            if state[1] >= self.grid_size:
                new_pos = state # if you are in the corridor you don't move
            else:
                new_pos = (min(self.grid_size-1, state[0]+1), state[1])
        else:
            raise ValueError("Invalid action.")
        return new_pos

    def state_to_index(self, state):
        if state[1] >= self.grid_size:
            return self.grid_size ** 2 + state[1] - self.grid_size
        return state[0] + state[1] * self.grid_size

    def index_to_state(self, index):
        """Converts an index to a state tuple (i, j)."""
        if index >= self.grid_size ** 2:
            i = self.grid_size - 1
            j = self.grid_size + index - self.grid_size ** 2
            return (i, j)
        i = index % self.grid_size
        j = index // self.grid_size
        #print("Converting %d => (%d, %d)" %(index, i, j))
        return (i, j)

    def sample_next_state(self, action):
        current_state = self.state_to_index(self.current_pos)
        action_probabilities = self.transition_matrix[:, current_state, action]
        next_state_index = np.random.choice(self.state_space.n, p=action_probabilities)
        next_pos = self.index_to_state(next_state_index)
        #print("Taking action %d from cur_pos index:%d (%d, %d): going to index %d which is state => (%d, %d)" %(action, current_state, self.current_pos[0], self.current_pos[1], next_state_index, next_pos[0], next_pos[1]))
        return next_pos

    def step(self, action):
        #print("Time-step: %d" %self.steps_taken)
        if self.done:
            raise ValueError("Episode has already ended.")
        new_pos = self.sample_next_state(action)
        reward = -0.1  # default reward for moving
        if new_pos == self.goal:
            reward = 1.0
            self.done = True
        self.current_pos = new_pos
        self.steps_taken += 1
        if self.time_horizon != -1 and self.steps_taken >= self.time_horizon:
            self.done = True
        return self.state_to_index(self.current_pos), reward, self.done, False, {}

    def reset(self, seed=None, options=None):

        if self.randomize == 1:
            self.current_pos = self.index_to_state(np.random.randint(0, self.state_space.n))
        else:
            self.current_pos = (0, 0)
        self.done = False
        self.steps_taken = 0
        return self.state_to_index(self.current_pos)

    def render(self, mode='human'):
        if mode == 'human':
            for j in range(self.grid_size):
                for i in range(self.grid_size):
                    if (i, j) == self.current_pos:
                        print("X ", end='')
                    elif (i, j) == self.goal:
                        print("G ", end='')
                    else:
                        print("_ ", end='')
                print()
            print()

In [ ]:
class GridworldEnvGoalless(GridworldEnv):
    '''
    This extension of the environment work functionally like the previous, but it removes the ending goal state.
    '''
    def step(self, action):
        #print("Time-step: %d" %self.steps_taken)
        if self.done:
            raise ValueError("Episode has already ended.")
        new_pos = self.sample_next_state(action)
        reward = -0.1  # default reward for moving
        if new_pos == self.goal:
            reward = 1.0
        self.current_pos = new_pos
        self.steps_taken += 1
        if self.time_horizon != -1 and self.steps_taken >= self.time_horizon:
            self.done = True
        return self.state_to_index(self.current_pos), reward, self.done, {}

# Reward Reinforce Agent MDP

In [ ]:
# Define the REINFORCE agent
class REINFORCEAgent():
    '''
    This class is the implementation of a REINFORCE agent that tries to maximize the objective function J(θ)=E_(τ ~ p_π)[R(τ)]
    Args:
     - env: the instance of the environment on which the agent is acting.
     - alpha: the value of the learning rate to compute the policy update.
     - gamma: the value of the discount for the reward in order to compute the discounted reward.
    '''
    def __init__(self, env, alpha=0.1, gamma=0.9):
        self.env = env
        self.alpha = alpha  # learning rate
        self.gamma = gamma  # discount factor
        self.policy_params = np.zeros((env.state_space.n, env.action_space.n))
        self.ohe_states = np.identity(env.state_space.n)

    def get_probability(self, state):
        # Compute the dot product
        params = np.dot(state, self.policy_params)
        # Compute the softmax
        probs = np.exp(params) / np.sum(np.exp(params))
        return probs

    def compute_returns(self, episode):
        G = 0
        returns = []
        for t in reversed(range(len(episode))):
            _, _, _, reward, _ = episode[t]
            G = self.gamma * G + reward
            returns.append(G)
        returns = np.array(list(reversed(returns)))
        return returns

    def get_action(self, state):
        # Get probability vector
        probs = self.get_probability(state)
        # Sample the action
        action = np.random.choice(len(probs), p=probs)
        return action, probs

    def update_single_sampling(self, episode):
        '''
        This version of the update is the Monte Carlo sampling version of the REINFORCE algorithm.

        Args:
         - episode: the sampled trajectory from which we compute the policy gradient.
        '''
        # Compute returns
        returns = self.compute_returns(episode)
        # Compute the policy gradient
        grad = np.zeros_like(self.policy_params)
        for t in range(len(episode)):
            state, action, probs, _, _ = episode[t]
            dlogp = np.zeros(self.env.action_space.n)
            for i in range(env.action_space.n):
                dlogp[i] = 1.0 - probs[i] if i == action else -probs[i]
            grad += np.outer(state, dlogp) * returns[t]
        # Update the policy parameters
        self.policy_params += self.alpha * grad

    def update_multiple_sampling(self, trajectories):
        '''
        This version of the update takes into consideration the approximation of the gradient by sampling multiple trajectories. Instead
        of working with only one trajectory it works with multiple trajectories in order to have a more accurate representation of the expected
        value of the ∇J(θ).

        Args:
         - trajectories: a list of sampled trajectories from which we compute the policy gradient.

        '''
        # Compute the policy gradient
        grad = np.zeros_like(self.policy_params)
        for episode in trajectories:
            returns = self.compute_returns(episode)
            for t in range(len(episode)):
                state, action, probs, _, _ = episode[t]
                dlogp = np.zeros(self.env.action_space.n)
                for i in range(env.action_space.n):
                    dlogp[i] = 1.0 - probs[i] if i == action else -probs[i]
                grad += np.outer(state, dlogp) * returns[t]
        grad /= len(trajectories)
        # Update the policy parameters
        self.policy_params += self.alpha * grad

# Entropy Reinforce Agent MDP


In [ ]:
class REINFORCEAgentE(REINFORCEAgent):
    '''
    This class is the extension of the previous Reinforce agent. The only change is the objective followed by the agent that this time is going
    to be the state-action visitation entropy J(θ) = E_(τ ~ p_π)[H(d_τ(s,a))] .

    Args:
         - policy:
            0: initialize the dummy policy;
            1: initialize the optimal policy;
            2: initialize the uniform policy.
    '''
    def __init__(self, env, alpha=0.1, gamma=0.9, policy=-1):
        super().__init__(env=env, alpha=alpha, gamma=gamma)
        # Optional initialization to have a non-uniform initial policy
        self.init_policy(policy)

    def init_policy(self, policy):
        if policy == 0:
            self.policy_params = self.create_dummy_policy()
        elif policy == 1:
            self.policy_params = self.create_optimal_policy()
        else:
            self.policy_params = np.ones((env.state_space.n, env.action_space.n))

    def create_dummy_policy(self):
        params = np.zeros((env.state_space.n, env.action_space.n))
        for i in range(self.env.state_space.n):
                params[i, 0] = 1
        return params

    def create_optimal_policy(self):
        params = np.zeros((env.state_space.n, env.action_space.n))
        k = 1
        actions = [3,3,3,2]
        for i in range(self.env.state_space.n):
                if i % self.env.grid_size == self.env.grid_size - 1 and (k == 1):
                    params[i, 1] = 500
                    k += 1
                elif i % self.env.grid_size == self.env.grid_size - 1 and (k == 3):
                    params[i, actions[k]] = 500
                    k = 0
                elif i % self.env.grid_size == 0 and k == 2:
                    params[i, 1] = 500
                    k += 1
                elif i % self.env.grid_size == 0 and k == 0:
                    params[i, actions[k]] = 500
                    k += 1
                else:
                    params[i, actions[k]] = 500
        return params

    def compute_entropy(self, d_t):
        # Create the condition to apply the log
        condition = d_t > 0
        # Compute log of d_t
        log_d_t = -np.log(d_t, where=condition)
        # Compute the entropy
        entropy = np.sum(np.multiply(d_t, log_d_t))
        return entropy

    def update_single_sampling(self, episode, d_t):
        # Compute entropy of d_t
        entropy = self.compute_entropy(d_t)
        # Update policy parameters using the gradient of the entropy of the t-step state distribution
        grad = np.zeros_like(self.policy_params)
        for t in range(len(episode)):
            state, action, probs, _, _ = episode[t]
            # Compute the policy gradient
            dlogp = np.zeros(self.env.action_space.n)
            for i in range(env.action_space.n):
                dlogp[i] = 1.0 - probs[i] if i == action else -probs[i]
            grad += np.outer(state, dlogp) * entropy
        grad *= entropy
        # Update the policy parameters
        self.policy_params += self.alpha * grad
        return entropy

    def update_multiple_sampling(self, trajectories):
        # Update policy parameters using the approximated gradient of the entropy objective function
        grad = np.zeros_like(self.policy_params)
        entropies = []
        for episode, d_t in trajectories:
            # Initialize the gradient of the k-th sampled trajectory
            grad_k = np.zeros_like(self.policy_params)
            # Compute entropy of d_t
            entropy = self.compute_entropy(d_t)
            # Compute the gradient
            for t in range(len(episode)):
                state, action, probs, _, _ = episode[t]
                # Compute the policy gradient
                dlogp = np.zeros(self.env.action_space.n)
                for i in range(env.action_space.n):
                    dlogp[i] = 1.0 - probs[i] if i == action else -probs[i]
                grad_k += np.outer(state, dlogp) * entropy
            # Sum the k-th gradient to the final gradient
            grad += grad_k
            entropies.append(entropy)
        # Divide the gradient by the number of trajectories sampled
        grad /= len(trajectories)
        # Update the policy parameters
        self.policy_params += self.alpha * grad
        return entropies

# Gridworld POMDP


In [ ]:
class GridworldPOMDPEnv(GridworldEnv):
    '''
    This class implements the extension of the upper Gridworld MDP, making it become a POMDP by inserting the observation mechanism.
    The observations here are modeled as a probability of returning the real state of the MDP, in a Gaussian fashion based on the distance of the observed state from the real current state.

    Args:
     - grid_size: the size in height and length of the grid, N of the NxN matrix.
     - time_horizon: the maximum number of time steps the agent can take to get to the goal. If set to -1 the time horizon is ∞.
     - prob: the probability with which the environment takes the chosen action. If set to 0 the actions taken by the agent are deterministic.
     - steepness: a parameter to control the steepness of the Gaussian distribution that models the observation probability from the real state. A higher value makes it steeper.
    '''
    def __init__(self, grid_size=5, time_horizon=-1, prob=0.1, randomize=0, steepness=15):
        # Initialize the underlying Gridworld MDP
        super().__init__(grid_size=grid_size, time_horizon=time_horizon, randomize=randomize, prob=prob)
        # Initialize all the POMDP specific variables
        self.observation_space = spaces.Discrete(self.state_space.n)
        self.steepness = steepness
        self.observation_matrix = self.build_observation_matrix()

    def build_observation_matrix(self):
        '''
        This method creates the observation matrix for our environment.
        The observation function indexes are to be used in order:
         - first param: true state s
         - second param: observation o
        '''
        # Initialize the observation function with zeros
        observation_matrix = np.zeros((self.state_space.n, self.state_space.n))
        # Calculate the variance of the Gaussian distribution based on the grid size
        variance = (self.grid_size + self.length_corridor // 2) ** 2

        for s_i in range(self.observation_space.n):
            for s in range(self.observation_space.n):
                # Calculate the distance between the observed position and the true state
                distance = np.linalg.norm(np.array(self.index_to_state(s_i)) - np.array(self.index_to_state(s)))
                # Assign probability based on Gaussian distribution with adjusted steepness
                observation_matrix[s_i][s] += np.exp(-self.steepness * distance**2 / (2 * variance))
            # Normalize the probabilities for each row
            observation_matrix[s_i] /= np.sum(observation_matrix[s_i])

        # Return the built matrix
        return observation_matrix

    def reset(self):
        # Call the upper reset of the environment
        super().reset()
        return self.state_to_index(self.current_pos)
        '''
        # Set the initial belief and give it to the agent
        initial_belief = self.observation_matrix[self.state_to_index(self.current_pos)]
        return initial_belief
        '''

    def step(self, action):
        # Save the true state
        true_state = self.current_pos
        # Make the step of the underlying MDP
        next_state, reward, done, info = super().step(action)
        # Get the index of the state
        next_state_index = self.state_to_index(next_state)
        # Get the observation probabilities for the state
        obs_prob = self.observation_matrix[next_state_index]
        # Sample the next observation from the probabilities
        obs = np.random.choice(self.observation_space.n, p=obs_prob)
        return obs, reward, done, False, true_state

In [ ]:

class GridworldPOMDPEnvGoalless(GridworldEnvGoalless):
    '''
    This extension of the environment work functionally like the previous, but it removes the ending goal state.
    In the POMDP case it also adds the observation mechanism.
    '''
    def __init__(self, grid_size=5, time_horizon=-1, prob=0.1, steepness=15):
        # Initialize the underlying Gridworld MDP
        super().__init__(grid_size=grid_size, time_horizon=time_horizon, prob=prob)
        # Initialize all the POMDP specific variables
        self.observation_space = spaces.Discrete(self.state_space.n)
        self.steepness = steepness
        self.observation_matrix = self.build_observation_matrix()

    def build_observation_matrix(self):
        '''
        This method creates the observation matrix for our environment.
        The observation function indexes are to be used in order:
         - first param: true state s
         - second param: observation o
        '''
        # Initialize the observation function with zeros
        observation_matrix = np.zeros((self.observation_space.n, self.observation_space.n))
        # Calculate the variance of the Gaussian distribution based on the grid size
        variance = (self.grid_size // 2) ** 2

        for s_i in range(self.observation_space.n):
            for s in range(self.observation_space.n):
                # Calculate the distance between the observed position and the true state
                distance = np.linalg.norm(np.array(self.index_to_state(s_i)) - np.array(self.index_to_state(s)))
                # Assign probability based on Gaussian distribution with adjusted steepness
                observation_matrix[s_i][s] += np.exp(-self.steepness * distance**2 / (2 * variance))
            # Normalize the probabilities for each row
            observation_matrix[s_i] /= np.sum(observation_matrix[s_i])

        # Return the built matrix
        return observation_matrix

    def reset(self):
        # Call the upper reset of the environment
        super().reset()
        return self.state_to_index(self.current_pos)
        '''
        # Set the initial belief and give it to the agent
        initial_belief = self.observation_matrix[self.state_to_index(self.current_pos)]
        return initial_belief
        '''

    def step(self, action):
        # Save the true state
        true_state = self.current_pos
        # Make the step of the underlying MDP
        next_state, reward, done, info = super().step(action)
        # Get the index of the state
        next_state_index = self.state_to_index(next_state)
        # Get the observation probabilities for the state
        obs_prob = self.observation_matrix[next_state_index]
        # Sample the next observation from the probabilities
        obs = np.random.choice(self.observation_space.n, p=obs_prob)
        return obs, reward, done, False, true_state

# Gridworld POMDP BiModal

In [ ]:
class GridworldPOMDPEnvBiModal(GridworldEnvGoalless):
    '''
    This class implements the extension of the upper Gridworld MDP, making it become a POMDP by inserting the observation mechanism.
    The observations here are modeled as a probability of returning the real state of the MDP, in a Gaussian fashion based on the distance of the observed state from the real current state.

    Args:
     - grid_size: the size in height and length of the grid, N of the NxN matrix.
     - time_horizon: the maximum number of time steps the agent can take to get to the goal. If set to -1 the time horizon is ∞.
     - prob: the probability with which the environment takes the chosen action. If set to 0 the actions taken by the agent are deterministic.
     - steepness: a parameter to control the steepness of the Gaussian distribution that models the observation probability from the real state. A higher value makes it steeper.
    '''
    def __init__(self, grid_size=5, time_horizon=-1, prob=0.1, randomize=0, steepness=15):
        # Initialize the underlying Gridworld MDP
        super().__init__(grid_size=grid_size, time_horizon=time_horizon, randomize=randomize, prob=prob)
        # Initialize all the POMDP specific variables
        self.shift_amount = 0.80
        self.observation_space = spaces.Discrete(self.state_space.n)
        self.steepness = steepness
        self.observation_matrix = self.build_observation_matrix()

    def build_observation_matrix(self):
        '''
        This method creates the observation matrix for our environment.
        The observation function indexes are to be used in order:
         - first param: true state s
         - second param: observation o
        '''
        # Initialize the observation function with zeros
        observation_matrix = np.zeros((self.state_space.n, self.state_space.n))
        # Calculate the variance of the Gaussian distribution based on the grid size
        variance = (self.grid_size // 2) ** 2

        for s_i in range(self.observation_space.n):
            for s in range(self.observation_space.n):
                # Calculate the distance between the observed position and the true state
                distance = np.linalg.norm(np.array(self.index_to_state(s_i)) - np.array(self.index_to_state(s)))
                # Calculate the probability for the left-shifted Gaussian
                prob_left = np.exp(-self.steepness * (distance + self.shift_amount) ** 2 / (2 * variance))
                # Calculate the probability for the right-shifted Gaussian
                prob_right = np.exp(-self.steepness * (distance - self.shift_amount) ** 2 / (2 * variance))
                # Assign probability as the sum of left-shifted and right-shifted Gaussians
                observation_matrix[s_i][s] += prob_left + prob_right

            # Normalize the probabilities for each row
            observation_matrix[s_i] /= np.sum(observation_matrix[s_i])

        # Return the built matrix
        return observation_matrix

    def reset(self, seed=None, options=None):
        # Call the upper reset of the environment
        super().reset()
        return self.state_to_index(self.current_pos)
        '''
        # Set the initial belief and give it to the agent
        initial_belief = self.observation_matrix[self.state_to_index(self.current_pos)]
        return initial_belief
        '''

    def step(self, action):
        # Save the true state
        true_state = self.current_pos
        # Make the step of the underlying MDP
        next_state, reward, done, info = super().step(action)
        # Get the index of the state
        next_state_index = self.state_to_index(next_state)
        # Get the observation probabilities for the state
        obs_prob = self.observation_matrix[next_state_index]
        # Sample the next observation from the probabilities
        obs = np.random.choice(self.observation_space.n, p=obs_prob)
        return obs, reward, done, False, true_state

In [ ]:
gym.envs.register(
    id='GridworldPOMDPEnvBiModal-v0',
    entry_point=GridworldPOMDPEnvBiModal
)

In [ ]:
def run_trajectory(agent, env):
    episode = []
    state = env.reset()
    done = False
    total_reward = 0
    while not done:
        action = agent.get_action(state)
        next_state, reward, done, _ = env.step(action)
        episode.append((state, action, reward, next_state))
        state = next_state
        total_reward += reward
    return episode, total_reward



# NMRL

# Vector Trial


In [ ]:
class REINFORCEAgentEPOMDP(REINFORCEAgentE):
    '''
    This calls implements the Reinforce Agent for the POMDP defined above.
    It is implemented as the extension of the former MDP agent but adds the belief state and what is concerned by it.
    '''
    def __init__(self, env, alpha=0.1, gamma=0.9, n_traj=50):
        super().__init__(env=env, alpha=alpha, gamma=gamma)
        self.beliefs = np.ones((n_traj, env.observation_space.n)) / env.observation_space.n
        self.n_expected_value = 30

    def get_actions(self):
        actions = []
        probs = []
        for state in self.beliefs:
            action, prob = self.get_action(state)
            actions.append(action)
            probs.append(prob)
        return actions, probs

    def belief_update(self, actions, observations):
        batch_size = len(actions)

        # Get the indices of the observed states
        obs_state_indices = np.array([self.env.state_to_index(obs) for obs in observations])

        # Get the observation probabilities for all states
        obs_probabilities = self.env.observation_matrix[:, obs_state_indices]

        # Calculate the numerator: Obs(o|s) * sum_s' (P(s|s',a) * belief_state[s'])
        numerators = obs_probabilities * np.sum(self.env.transition_matrix[:, :, actions] * self.belief_state, axis=1)[:, np.newaxis]

        # Calculate the denominator: sum_s' (Obs(o|s') * sum_s'' (P(s'|s'',a) * belief_state[s'']))
        denominators = np.sum(obs_probabilities * np.sum(self.env.transition_matrix[:, :, actions] * self.belief_state, axis=1), axis=1)

        # Update the belief states for each trajectory in the batch
        for i in range(batch_size):
            self.belief_state[i] = numerators[i] / denominators[i]

    def get_states(self):
        '''
        This method is used to sample or compute the expected state given the current beliefs.
        Args:
        - behaviour: if 0, it returns the state from a single sampling.
                     if 1, it returns the state from multiple samplings.
                     No other values allowed.
        '''
        states = []
        for state in self.beliefs:
            states.append(self.get_state(state, 1))
        return states

    def get_state(self, belief, behaviour):
        '''
        This method is used to sample or compute the expected state given the current belief.
        Args:
         - behaviour: if 0 it returns the state from a single sampling.
                 if 1 it returns the state from multiple samplings.
                 No other values allowed.
        '''
        if behaviour != 0 and behaviour != 1:
            raise Exception("You have to pass me 0 or 1, read :/")

        if behaviour == 0:
            state = np.random.choice(belief.size, p=belief)
            state = env.index_to_state(state)
            return state

        if behaviour == 1:
            states = np.random.choice(belief.size, self.n_expected_value, p=belief)
            state = mode(states).mode[0]
            state = env.index_to_state(state)
            return state

In [ ]:
from gym.utils.step_api_compatibility import step_to_new_api
# Number of episodes for the training
n_episodes = 8000
n_traj = 50
# Number of runs per episode
n_run = 4
# Define the arguments for each environment
time_horizon = 30
steepness = 15
prob = 0
envs = gym.vector.make('GridworldPOMDPEnvBiModal-v0', time_horizon=time_horizon, steepness=steepness, prob=prob, num_envs=n_traj, new_step_api=True)
env = GridworldPOMDPEnvBiModal(time_horizon = time_horizon, steepness=15, prob=0)

with tqdm(total=n_run * n_episodes, ncols=80) as pbar:
    # Train the agent and plot the entropies
    list_entropies = []
    list_true_entropies = []
    for r in range(n_run):
        agent = REINFORCEAgentEPOMDP(env, alpha=0.5, n_traj=n_traj)  # Modify the agent to accept the parallel environments
        avg_entropies = []
        avg_true_entropies = []
        for i in range(n_episodes):
            episode = []
            d_t = np.zeros(env.observation_space.n)  # Initialize visitation counts for all parallel environments
            true_d_t = np.zeros(env.observation_space.n)
            envs.reset()
            done = np.zeros(n_traj, dtype=bool)
            while not np.all(done):
                #print(agent.beliefs)
                # Sample action and get probabilities from the belief
                actions, probs = agent.get_actions()
                # Sample state
                sampled_states = agent.get_states()
                # Take a step in the parallel environments
                print(actions)
                next_obs, rewards, done, _, true_states = envs.step(actions)
                # Get the indices of the states for all parallel environments
                state_indices = [env.state_to_index(state) for state in sampled_states]
                true_state_indices = [env.state_to_index(state) for state in true_states]
                # Update state visitation counts for all parallel environments
                for i, state_index in enumerate(state_indices):
                    d_t[state_index] += 1
                for i, true_state_index in enumerate(true_state_indices):
                    true_d_t[true_state_index] += 1
                episode.append((agent.belief_states, actions, probs, rewards, true_states))
                agent.belief_update(actions, next_obs)
            # Compute true entropy of the trajectory for all parallel environments
            true_d_t /= len(episode)
            true_entropies.append(agent.compute_entropy(true_d_t))
            d_t /= len(episode)
            episodes.append((episode, d_t))
            entropies = agent.update_multiple_sampling(episodes)
            avg_entropies.append(np.mean(entropies))
            avg_true_entropies.append(np.mean(true_entropies))
            pbar.update(1)
        agent.print_visuals(envs=envs, n_traj=n_traj)  # Modify the agent to accept the parallel environments
        list_entropies.append(avg_entropies)
        list_true_entropies.append(avg_true_entropies)
    list_entropies = np.transpose(np.array(list_entropies), (1, 0))
    list_true_entropies = np.transpose(np.array(list_true_entropies), (1, 0))

plot_graph(n_run, n_episodes, list_entropies, list_true_entropies, confidence)


  0%|                                                 | 0/32000 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:190: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed `return_info` to return information from the environment resetting.
  logger.warn(
<ipython-input-11-696a85eaaada>:70: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  state = mode(states).mode[0]
  0%|                                                 | 0/32000 [00:00<?, ?it/s]


[1, 3, 3, 1, 2, 3, 2, 1, 1, 2, 3, 3, 3, 0, 3, 3, 1, 1, 1, 0, 2, 0, 3, 2, 1, 0, 2, 1, 0, 0, 2, 3, 3, 1, 1, 3, 2, 3, 2, 1, 2, 0, 1, 2, 3, 2, 0, 1, 3, 3]
False
None


TypeError: ignored